## Análise exploratória visando calcular o percentual de pessoas acolhidas pelo Programa Família Acolhedora por município

In [1]:
import pandas as pd
import plotly.express as px
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
import numpy as np
import base64
from io import BytesIO

/Users/rislamiranda/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Tratamento da base referente às unidades de acolhimento

In [2]:
# Criar um dataframe com as unidades de acolhimento
df_unidade_acolhimento = pd.read_csv('../data/unidades_acolhimento/Censo_SUAS_2023_Unidade_Acolhimento_Dados_Gerais.csv', sep=';', encoding='latin1', low_memory=False)

In [3]:
df_unidade_acolhimento.head()

,NU_IDENTIFICADOR,IBGE,q0_1,q0_2,q0_3,q0_4,q0_5,q0_6,q0_7,q0_8,...,D29_1,D29_2,d46_1_atend_categoria,d46_2_quarto_usuario,d46_3_quarto_cuidador,d46_4_banheiros_usuario,d46_5_banheiros,q47_1_sala_adm,q47_2_sala_reun,q47_3_sala_atendimento
0,1100013302478,110001,UNIDADE DE ACOLHIMENTO CASA DA CRIANÇA DOM IGN...,Avenida,RIO GRANDE DO SUL,4982,,CENTRO,,76954-000,...,,,1 sala,De 4 a 5 quartos,nenhum quarto,De 2 a 3 banheiros,1 banheiro,1 sala,1 sala,1 sala
1,1100023302479,110002,UNIDADE DE ACOLHIMENTO CASA DA CRIANÇA FRANCIS...,Rua,Rua Flor do Ipê,2321,,setor 04,Próximo a APAE,76873-462,...,,,1 sala,De 4 a 5 quartos,nenhum quarto,De 2 a 3 banheiros,1 banheiro,De 2 a 3 salas,Nenhuma sala,1 sala
2,1100023303854,110002,UNIDADE DE ACOLHIMENTO ASSOCIAÇÃO ANGLICANA DE...,Rua,Pioneiro André Ribeiro,1628,,SETOR 02,,76873-224,...,,,1 sala,De 2 a 3 quartos,nenhum quarto,De 2 a 3 banheiros,1 banheiro,1 sala,1 sala,1 sala
3,1100023304841,110002,UNIDADE DE ACOLHIMENTO CENTRO ESPIRITA ALAAN K...,Rua,canario,1091,,setor 02,,76876-125,...,,,1 sala,8 ou mais quartos,nenhum quarto,8 ou mais banheiros,De 2 a 3 banheiros,1 sala,1 sala,1 sala
4,1100023310844,110002,UNIDADE DE ACOLHIMENTO - CASA DE PASSAGEM AMOREVI,Rua,guanambi,987,,setor 02,,76873-050,...,,,De 2 a 3 salas,De 6 a 7 quartos,De 2 a 3 quartos,De 4 a 5 banheiros,De 4 a 5 banheiros,De 2 a 3 salas,De 2 a 3 salas,De 2 a 3 salas


In [4]:
#df_unidade_acolhimento.columns.tolist()

In [5]:
df_unidade_acolhimento['NU_IDENTIFICADOR'].apply(type).unique()

array([<class 'int'>], dtype=object)

In [6]:
# Criar uma lista com nomes para as colunas, de forma a compreender melhor os dados
rename_cols_uni = {
    'q11':'qtd_pessoas_acolhidas',
    'q14_1_1':'qtd_masc_0_2',
    'q14_1_2':'qtd_masc_3_5',
    'q14_1_3':'qtd_masc_6_11',
    'q14_1_4':'qtd_masc_12_13',
    'q14_1_5':'qtd_masc_14_15',
    'q14_1_6':'qtd_masc_16_17',
    'q14_1_7':'qtd_masc_18_21',    
    'q14_1_8':'qtd_masc_22_59',
    'q14_1_9':'qtd_masc_60_79',
    'q14_1_10':'qtd_masc_acima_80',      
    'q14_1_11':'qtd_masc_total',
    'q14_2_1':'qtd_fem_0_2',
    'q14_2_2':'qtd_fem_3_5',
    'q14_2_3':'qtd_fem_6_11',
    'q14_2_4':'qtd_fem_12_13',
    'q14_2_5':'qtd_fem_14_15',
    'q14_2_6':'qtd_fem_16_17',
    'q14_2_7':'qtd_fem_18_21',    
    'q14_2_8':'qtd_fem_22_59',
    'q14_2_9':'qtd_fem_60_79',
    'q14_2_10':'qtd_fem_acima_80', 
    'q14_2_11':'qtd_fem_total',      
}

In [7]:
# Renomear as colunas do dataframe de unidades de acolhimento de acordo com a lista criada
df_unidade_acolhimento = df_unidade_acolhimento.rename(columns=rename_cols_uni)

In [8]:
# Criar uma lista para filtrar o dataframe de unidades acolhedoras
filter_pessoas_uni = ['NU_IDENTIFICADOR',
    'IBGE',
    'qtd_pessoas_acolhidas',
    'qtd_masc_0_2',
    'qtd_masc_3_5',
    'qtd_masc_6_11',
    'qtd_masc_12_13',
    'qtd_masc_14_15',
    'qtd_masc_16_17',
    'qtd_masc_18_21',    
    'qtd_masc_22_59',
    'qtd_masc_60_79',
    'qtd_masc_acima_80',      
    'qtd_masc_total',
    'qtd_fem_0_2',
    'qtd_fem_3_5',
    'qtd_fem_6_11',
    'qtd_fem_12_13',
    'qtd_fem_14_15',
    'qtd_fem_16_17',
    'qtd_fem_18_21',    
    'qtd_fem_22_59',
    'qtd_fem_60_79',
    'qtd_fem_acima_80', 
    'qtd_fem_total']

In [9]:
# Criar um dataframe a partir do filtro de pessoas acolhidas
df_pessoas_uni_acolhi = df_unidade_acolhimento[filter_pessoas_uni]

In [10]:
df_pessoas_uni_acolhi.head()

,NU_IDENTIFICADOR,IBGE,qtd_pessoas_acolhidas,qtd_masc_0_2,qtd_masc_3_5,qtd_masc_6_11,qtd_masc_12_13,qtd_masc_14_15,qtd_masc_16_17,qtd_masc_18_21,...,qtd_fem_3_5,qtd_fem_6_11,qtd_fem_12_13,qtd_fem_14_15,qtd_fem_16_17,qtd_fem_18_21,qtd_fem_22_59,qtd_fem_60_79,qtd_fem_acima_80,qtd_fem_total
0,1100013302478,110001,6,1,1,0,1,0,1,0,...,0,0,2,0,0,0,0,0,0,2
1,1100023302479,110002,18,0,2,6,0,0,0,0,...,0,5,1,1,1,0,0,0,0,10
2,1100023303854,110002,6,0,0,0,0,1,0,0,...,1,0,0,0,0,0,3,0,0,5
3,1100023304841,110002,21,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,2,3
4,1100023310844,110002,21,0,1,1,1,1,0,0,...,0,0,0,0,0,0,9,0,0,9


In [11]:
df_pessoas_uni_acolhi.shape

(6605, 25)

In [12]:
df_pessoas_uni_acolhi.columns

Index(['NU_IDENTIFICADOR', 'IBGE', 'qtd_pessoas_acolhidas', 'qtd_masc_0_2',
       'qtd_masc_3_5', 'qtd_masc_6_11', 'qtd_masc_12_13', 'qtd_masc_14_15',
       'qtd_masc_16_17', 'qtd_masc_18_21', 'qtd_masc_22_59', 'qtd_masc_60_79',
       'qtd_masc_acima_80', 'qtd_masc_total', 'qtd_fem_0_2', 'qtd_fem_3_5',
       'qtd_fem_6_11', 'qtd_fem_12_13', 'qtd_fem_14_15', 'qtd_fem_16_17',
       'qtd_fem_18_21', 'qtd_fem_22_59', 'qtd_fem_60_79', 'qtd_fem_acima_80',
       'qtd_fem_total'],
      dtype='object')

In [13]:
print(df_pessoas_uni_acolhi[['NU_IDENTIFICADOR', 'IBGE', 'qtd_pessoas_acolhidas', 'qtd_masc_0_2',
       'qtd_masc_3_5', 'qtd_masc_6_11', 'qtd_masc_12_13', 'qtd_masc_14_15',
       'qtd_masc_16_17', 'qtd_masc_18_21', 'qtd_masc_22_59', 'qtd_masc_60_79',
       'qtd_masc_acima_80', 'qtd_masc_total', 'qtd_fem_0_2', 'qtd_fem_3_5',
       'qtd_fem_6_11', 'qtd_fem_12_13', 'qtd_fem_14_15', 'qtd_fem_16_17',
       'qtd_fem_18_21', 'qtd_fem_22_59', 'qtd_fem_60_79', 'qtd_fem_acima_80',
       'qtd_fem_total']].dtypes)

NU_IDENTIFICADOR          int64
IBGE                      int64
qtd_pessoas_acolhidas    object
qtd_masc_0_2             object
qtd_masc_3_5             object
qtd_masc_6_11            object
qtd_masc_12_13           object
qtd_masc_14_15           object
qtd_masc_16_17           object
qtd_masc_18_21           object
qtd_masc_22_59           object
qtd_masc_60_79           object
qtd_masc_acima_80        object
qtd_masc_total           object
qtd_fem_0_2              object
qtd_fem_3_5              object
qtd_fem_6_11             object
qtd_fem_12_13            object
qtd_fem_14_15            object
qtd_fem_16_17            object
qtd_fem_18_21            object
qtd_fem_22_59            object
qtd_fem_60_79            object
qtd_fem_acima_80         object
qtd_fem_total            object
dtype: object


In [14]:
# As colunas são tipo object o que significa que elas têm tipo string no meio o que vai impactar na soma

In [15]:
# Fazer uma cópia explícita do DataFrame (opcional, mas recomendado)
df_pessoas_uni_acolhi = df_pessoas_uni_acolhi.copy()

# Converter colunas para tipo numérico e depois para inteiro
cols_para_converter = [
    'qtd_pessoas_acolhidas', 'qtd_masc_0_2',
    'qtd_masc_3_5', 'qtd_masc_6_11', 'qtd_masc_12_13', 'qtd_masc_14_15',
    'qtd_masc_16_17', 'qtd_masc_18_21', 'qtd_masc_22_59', 'qtd_masc_60_79',
    'qtd_masc_acima_80', 'qtd_masc_total', 'qtd_fem_0_2', 'qtd_fem_3_5',
    'qtd_fem_6_11', 'qtd_fem_12_13', 'qtd_fem_14_15', 'qtd_fem_16_17',
    'qtd_fem_18_21', 'qtd_fem_22_59', 'qtd_fem_60_79', 'qtd_fem_acima_80',
    'qtd_fem_total'
]

In [16]:
# Limpeza adicional das colunas
for col in cols_para_converter:
    df_pessoas_uni_acolhi[col] = (
        df_pessoas_uni_acolhi[col]
        .astype(str)  # Converter para string
        .str.strip()  # Remover espaços em branco no início e fim
        .str.replace(r'\D', '', regex=True)  # Remover qualquer caractere que não seja dígito
        .replace('', '0')  # Substituir strings vazias por '0'
    )

# Converter para numérico e depois para inteiro
df_pessoas_uni_acolhi[cols_para_converter] = (
    df_pessoas_uni_acolhi[cols_para_converter]
    .apply(pd.to_numeric, errors='coerce')
    .fillna(0)
    .astype(int)
)

# Verificar os tipos de dados após a conversão
print(df_pessoas_uni_acolhi[cols_para_converter].dtypes)

qtd_pessoas_acolhidas    int64
qtd_masc_0_2             int64
qtd_masc_3_5             int64
qtd_masc_6_11            int64
qtd_masc_12_13           int64
qtd_masc_14_15           int64
qtd_masc_16_17           int64
qtd_masc_18_21           int64
qtd_masc_22_59           int64
qtd_masc_60_79           int64
qtd_masc_acima_80        int64
qtd_masc_total           int64
qtd_fem_0_2              int64
qtd_fem_3_5              int64
qtd_fem_6_11             int64
qtd_fem_12_13            int64
qtd_fem_14_15            int64
qtd_fem_16_17            int64
qtd_fem_18_21            int64
qtd_fem_22_59            int64
qtd_fem_60_79            int64
qtd_fem_acima_80         int64
qtd_fem_total            int64
dtype: object


In [17]:
df_pessoas_uni_acolhi = df_pessoas_uni_acolhi.copy()

# Criar a coluna 'qtd_masc_acima_18' somando as faixas etárias acima de 18 anos para masculino
df_pessoas_uni_acolhi['qtd_masc_acima_18'] = (
    df_pessoas_uni_acolhi[['qtd_masc_18_21', 'qtd_masc_22_59', 'qtd_masc_60_79', 'qtd_masc_acima_80']]
    .sum(axis=1)
)

In [18]:
# Criar a coluna 'qtd_fem_acima_18' somando as faixas etárias acima de 18 anos para feminino
df_pessoas_uni_acolhi['qtd_fem_acima_18'] = (
    df_pessoas_uni_acolhi[['qtd_fem_18_21', 'qtd_fem_22_59', 'qtd_fem_60_79', 'qtd_fem_acima_80']]
    .sum(axis=1)
)

In [19]:
# Criar uma cópia explícita para evitar o SettingWithCopyWarning
df_verificacao = df_pessoas_uni_acolhi[['qtd_masc_18_21', 'qtd_masc_22_59', 'qtd_masc_60_79', 'qtd_masc_acima_80', 'qtd_masc_acima_18']].copy()

# Calcular a soma manualmente usando .loc[]
df_verificacao.loc[:, 'soma_manual'] = df_verificacao[['qtd_masc_18_21', 'qtd_masc_22_59', 'qtd_masc_60_79', 'qtd_masc_acima_80']].sum(axis=1)

# Calcular a diferença usando .loc[]
df_verificacao.loc[:, 'diferenca'] = df_verificacao['qtd_masc_acima_18'] - df_verificacao['soma_manual']

# Exibir as primeiras linhas
print(df_verificacao.head())

   qtd_masc_18_21  qtd_masc_22_59  qtd_masc_60_79  qtd_masc_acima_80  \
0               0               0               0                  0   
1               0               0               0                  0   
2               0               0               0                  0   
3               0               0              11                  7   
4               0               6               2                  0   

   qtd_masc_acima_18  soma_manual  diferenca  
0                  0            0          0  
1                  0            0          0  
2                  0            0          0  
3                 18           18          0  
4                  8            8          0  


In [20]:
print(df_pessoas_uni_acolhi.columns.tolist())

['NU_IDENTIFICADOR', 'IBGE', 'qtd_pessoas_acolhidas', 'qtd_masc_0_2', 'qtd_masc_3_5', 'qtd_masc_6_11', 'qtd_masc_12_13', 'qtd_masc_14_15', 'qtd_masc_16_17', 'qtd_masc_18_21', 'qtd_masc_22_59', 'qtd_masc_60_79', 'qtd_masc_acima_80', 'qtd_masc_total', 'qtd_fem_0_2', 'qtd_fem_3_5', 'qtd_fem_6_11', 'qtd_fem_12_13', 'qtd_fem_14_15', 'qtd_fem_16_17', 'qtd_fem_18_21', 'qtd_fem_22_59', 'qtd_fem_60_79', 'qtd_fem_acima_80', 'qtd_fem_total', 'qtd_masc_acima_18', 'qtd_fem_acima_18']


In [21]:
colunas_para_remover = [
    'NU_IDENTIFICADOR', 'qtd_masc_18_21', 'qtd_masc_22_59', 'qtd_masc_60_79',
    'qtd_masc_acima_80', 'qtd_fem_18_21', 'qtd_fem_22_59', 'qtd_fem_60_79', 'qtd_fem_acima_80'
]

# Verificar quais colunas estão ausentes
colunas_existentes = [col for col in colunas_para_remover if col in df_pessoas_uni_acolhi.columns]
colunas_ausentes = [col for col in colunas_para_remover if col not in df_pessoas_uni_acolhi.columns]

print("Colunas existentes:", colunas_existentes)
print("Colunas ausentes:", colunas_ausentes)

Colunas existentes: ['NU_IDENTIFICADOR', 'qtd_masc_18_21', 'qtd_masc_22_59', 'qtd_masc_60_79', 'qtd_masc_acima_80', 'qtd_fem_18_21', 'qtd_fem_22_59', 'qtd_fem_60_79', 'qtd_fem_acima_80']
Colunas ausentes: []


In [22]:
df_pessoas_uni_acolhi = df_pessoas_uni_acolhi.drop(columns=colunas_existentes)

In [23]:
df_pessoas_uni_acolhi.head()

,IBGE,qtd_pessoas_acolhidas,qtd_masc_0_2,qtd_masc_3_5,qtd_masc_6_11,qtd_masc_12_13,qtd_masc_14_15,qtd_masc_16_17,qtd_masc_total,qtd_fem_0_2,qtd_fem_3_5,qtd_fem_6_11,qtd_fem_12_13,qtd_fem_14_15,qtd_fem_16_17,qtd_fem_total,qtd_masc_acima_18,qtd_fem_acima_18
0,110001,6,1,1,0,1,0,1,4,0,0,0,2,0,0,2,0,0
1,110002,18,0,2,6,0,0,0,8,2,0,5,1,1,1,10,0,0
2,110002,6,0,0,0,0,1,0,1,1,1,0,0,0,0,5,0,3
3,110002,21,0,0,0,0,0,0,18,0,0,0,0,0,0,3,18,3
4,110002,21,0,1,1,1,1,0,12,0,0,0,0,0,0,9,8,9


### Tratamento da base referente ao Programa Família Acolhedora

In [24]:
df_familia_acolhedora = pd.read_csv('../data/familia_acolhedora/Censo_SUAS_2023_Familia_Acolhedora_Dados_Gerais 1.csv', sep=';', encoding='latin1', low_memory=False)

In [25]:
df_familia_acolhedora.head()

,IBGE,NU_IDENTIFICADOR,q0_1,q0_2,q0_3,q0_4,q0_5,q0_6,q0_7,q0_8,...,q44_12_1_velha,q44_12_2_velha,q44_12_3_velha,q44_12_4_velha,q44_12_5_velha,q44_12_6_velha,q44_12_7_velha,q44_12_8_velha,q44_12_9_velha,d12
0,110011,"1,10011E+12",UNIDADE DE ACOLHIMENTO EM FAMÍLIA ACOLHEDORA,3,RIO BRANCO,1444,NaN,SETOR 02,NaN,76890-000,...,1,0,0,0,0,0,0,0,0,2023
1,110012,"1,10012E+12",UNIDADE DE ACOLHIMENTO FAMILIAR EM FAMÍLIA AC...,32,Miguel Galdino,85,NaN,Jardim dos Migrantes,NaN,76900-804,...,0,0,0,0,0,0,0,0,1,2022
2,110020,"1,1002E+12",UNIDADE DE ACOLHIMENTO,32,Geraldo Ferreira,135,CREAS,Agenor de Carvalho,CREAS,76820-316,...,0,0,0,0,0,0,0,0,0,De 2016 a 2019
3,120040,"1,2004E+12",UNIDADE DE ACOLHIMENTO SERVIÇO DE ACOLHIMENTO ...,32,AMOTY PASCOAL,110,NaN,RUY LINO,NaN,69917-405,...,0,0,0,0,0,0,0,0,0,De 2011 a 2015
4,130170,"1,3017E+12",Unidade de Acolhimento,32,Pará,0,NaN,São Crsitóvão,NaN,69800-000,...,0,0,0,0,0,0,0,0,1,De 2020 a 2021


In [26]:
# Verificou-se que a coluna NU_IDENTIFICADOR - que serve de link para as demais tabelas do Censo - no dataframe da família acolhedora
# está como string (texto)
df_familia_acolhedora['NU_IDENTIFICADOR'].apply(type).unique()

array([<class 'str'>], dtype=object)

In [27]:
# Criar um regex para retirar espaços vazios e as vírgulas para transformar a coluna em tipo inteiro
df_familia_acolhedora['NU_IDENTIFICADOR'] = df_familia_acolhedora['NU_IDENTIFICADOR'].str.replace(',', '.', regex=False).astype(float).astype(int)

In [28]:
df_familia_acolhedora['NU_IDENTIFICADOR'].apply(type).unique()

array([<class 'int'>], dtype=object)

In [29]:
# Criar uma lista com nomes para as colunas, de forma a compreender melhor os dados
rename_cols_fam = {
    'q18':'qtd_criancas_acolhidas',
    'q20_1_1':'qtd_masc_0_2',
    'q20_1_2':'qtd_masc_3_5',
    'q20_1_3':'qtd_masc_6_11',
    'q20_1_4':'qtd_masc_12_13',
    'q20_1_5':'qtd_masc_14_15',
    'q20_1_6':'qtd_masc_16_17',
    'q20_1_8':'qtd_masc_acima_18',
    'q20_1_9':'qtd_masc_total',
    'q20_2_1':'qtd_fem_0_2',
    'q20_2_2':'qtd_fem_3_5',
    'q20_2_3':'qtd_fem_6_11',
    'q20_2_4':'qtd_fem_12_13',
    'q20_2_5':'qtd_fem_14_15',
    'q20_2_6':'qtd_fem_16_17',
    'q20_2_8':'qtd_fem_acima_18',
    'q20_2_9':'qtd_fem_total',
}

In [30]:
# Renomear as colunas do dataframe de família acolhedora de acordo com a lista criada
df_familia_acolhedora = df_familia_acolhedora.rename(columns=rename_cols_fam)

In [31]:
# Criar uma lista para filtrar o dataframe de famílias acolhedoras
filter_criancas_fam = ['NU_IDENTIFICADOR',
    'IBGE',
    'qtd_criancas_acolhidas',
    'qtd_masc_0_2',
    'qtd_masc_3_5',
    'qtd_masc_6_11',
    'qtd_masc_12_13',
    'qtd_masc_14_15',
    'qtd_masc_16_17',
    'qtd_masc_acima_18',
    'qtd_masc_total',
    'qtd_fem_0_2',
    'qtd_fem_3_5',
    'qtd_fem_6_11',
    'qtd_fem_12_13',
    'qtd_fem_14_15',
    'qtd_fem_16_17',
    'qtd_fem_acima_18',
    'qtd_fem_total',
]

In [32]:
# Criar um datafarme a partir do filtro de crianças/adolescentes acolhidas
df_pessoas_fam_acolhedora = df_familia_acolhedora[filter_criancas_fam]

In [33]:
df_pessoas_fam_acolhedora.head()

,NU_IDENTIFICADOR,IBGE,qtd_criancas_acolhidas,qtd_masc_0_2,qtd_masc_3_5,qtd_masc_6_11,qtd_masc_12_13,qtd_masc_14_15,qtd_masc_16_17,qtd_masc_acima_18,qtd_masc_total,qtd_fem_0_2,qtd_fem_3_5,qtd_fem_6_11,qtd_fem_12_13,qtd_fem_14_15,qtd_fem_16_17,qtd_fem_acima_18,qtd_fem_total
0,1100110000000,110011,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1100120000000,110012,5,0,1,2,0,1,0,0,4,1,0,0,0,0,0,0,1
2,1100200000000,110020,5,0,0,0,2,0,0,0,2,1,0,0,0,1,1,0,3
3,1200400000000,120040,9,0,0,0,0,0,0,0,0,2,2,5,0,0,0,0,9
4,1301700000000,130170,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [34]:
df_pessoas_fam_acolhedora.shape

(620, 19)

In [35]:
df_pessoas_fam_acolhedora.columns

Index(['NU_IDENTIFICADOR', 'IBGE', 'qtd_criancas_acolhidas', 'qtd_masc_0_2',
       'qtd_masc_3_5', 'qtd_masc_6_11', 'qtd_masc_12_13', 'qtd_masc_14_15',
       'qtd_masc_16_17', 'qtd_masc_acima_18', 'qtd_masc_total', 'qtd_fem_0_2',
       'qtd_fem_3_5', 'qtd_fem_6_11', 'qtd_fem_12_13', 'qtd_fem_14_15',
       'qtd_fem_16_17', 'qtd_fem_acima_18', 'qtd_fem_total'],
      dtype='object')

In [36]:
print(df_pessoas_fam_acolhedora[['NU_IDENTIFICADOR', 'IBGE', 'qtd_criancas_acolhidas', 'qtd_masc_0_2',
       'qtd_masc_3_5', 'qtd_masc_6_11', 'qtd_masc_12_13', 'qtd_masc_14_15',
       'qtd_masc_16_17', 'qtd_masc_acima_18', 'qtd_masc_total', 'qtd_fem_0_2',
       'qtd_fem_3_5', 'qtd_fem_6_11', 'qtd_fem_12_13', 'qtd_fem_14_15',
       'qtd_fem_16_17', 'qtd_fem_acima_18', 'qtd_fem_total']].dtypes)

NU_IDENTIFICADOR          int64
IBGE                      int64
qtd_criancas_acolhidas    int64
qtd_masc_0_2              int64
qtd_masc_3_5              int64
qtd_masc_6_11             int64
qtd_masc_12_13            int64
qtd_masc_14_15            int64
qtd_masc_16_17            int64
qtd_masc_acima_18         int64
qtd_masc_total            int64
qtd_fem_0_2               int64
qtd_fem_3_5               int64
qtd_fem_6_11              int64
qtd_fem_12_13             int64
qtd_fem_14_15             int64
qtd_fem_16_17             int64
qtd_fem_acima_18          int64
qtd_fem_total             int64
dtype: object


### Analisar a porcentagem das pessoas acolhidas

In [39]:
# Carregar os dados de município de acordo com o código do IBGE
df_municipios = pd.read_excel('../data/dados_geo/RELATORIO_DTB_BRASIL_MUNICIPIO.xls', header=6, engine='xlrd')

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
*** No CODEPAGE record, no encoding_override: will use 'iso-8859-1'


In [40]:
df_municipios.head()

,UF,Nome_UF,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município
0,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,1101,Porto Velho,110002,Ariquemes,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso
3,11,Rondônia,1102,Ji-Paraná,110004,Ji-Paraná,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste
4,11,Rondônia,1101,Porto Velho,110002,Ariquemes,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes


In [41]:
df_municipios.columns

Index(['UF', 'Nome_UF', 'Região Geográfica Intermediária',
       'Nome Região Geográfica Intermediária', 'Região Geográfica Imediata',
       'Nome Região Geográfica Imediata', 'Mesorregião Geográfica',
       'Nome_Mesorregião', 'Microrregião Geográfica', 'Nome_Microrregião',
       'Município', 'Código Município Completo', 'Nome_Município'],
      dtype='object')

In [42]:
df_pessoas_uni_acolhi.columns

Index(['IBGE', 'qtd_pessoas_acolhidas', 'qtd_masc_0_2', 'qtd_masc_3_5',
       'qtd_masc_6_11', 'qtd_masc_12_13', 'qtd_masc_14_15', 'qtd_masc_16_17',
       'qtd_masc_total', 'qtd_fem_0_2', 'qtd_fem_3_5', 'qtd_fem_6_11',
       'qtd_fem_12_13', 'qtd_fem_14_15', 'qtd_fem_16_17', 'qtd_fem_total',
       'qtd_masc_acima_18', 'qtd_fem_acima_18'],
      dtype='object')

In [43]:
df_pessoas_fam_acolhedora.columns

Index(['NU_IDENTIFICADOR', 'IBGE', 'qtd_criancas_acolhidas', 'qtd_masc_0_2',
       'qtd_masc_3_5', 'qtd_masc_6_11', 'qtd_masc_12_13', 'qtd_masc_14_15',
       'qtd_masc_16_17', 'qtd_masc_acima_18', 'qtd_masc_total', 'qtd_fem_0_2',
       'qtd_fem_3_5', 'qtd_fem_6_11', 'qtd_fem_12_13', 'qtd_fem_14_15',
       'qtd_fem_16_17', 'qtd_fem_acima_18', 'qtd_fem_total'],
      dtype='object')

In [44]:
# Contar a frequência dos valores na coluna 'IBGE'
contagem_ibge = df_pessoas_uni_acolhi['IBGE'].value_counts()

# Exibir a contagem
print(contagem_ibge)

IBGE
355030    354
310620    102
330455     91
431490     69
410690     69
         ... 
231230      1
231330      1
330290      1
330280      1
110001      1
Name: count, Length: 2300, dtype: int64


#### Criar um dataframe que agrupa as quantidades total e de gênero de pessoas acolhidas por município

In [45]:
df_qtd_total_uni = df_pessoas_uni_acolhi.groupby('IBGE', as_index=False)[['qtd_pessoas_acolhidas', 'qtd_masc_total', 'qtd_fem_total']].sum()

In [46]:
df_qtd_total_uni.head()

,IBGE,qtd_pessoas_acolhidas,qtd_masc_total,qtd_fem_total
0,11,25,21,4
1,12,15,12,3
2,13,74,27,47
3,14,107,68,39
4,15,96,54,42


In [47]:
df_qtd_uni_munic = df_qtd_total_uni[df_qtd_total_uni['IBGE'].between(100000, 999999)]

In [48]:
df_qtd_uni_munic.head()

,IBGE,qtd_pessoas_acolhidas,qtd_masc_total,qtd_fem_total
22,110001,6,4,2
23,110002,66,39,27
24,110003,4,2,2
25,110004,91,67,24
26,110005,4,3,1


In [49]:
# Fazer uma cópia explícita do DataFrame
df_qtd_uni_munic = df_qtd_uni_munic.copy()

# Padronizar a coluna IBGE no DataFrame 'df_qtd_total_municipios' para ter seis dígitos
df_qtd_uni_munic['IBGE'] = df_qtd_uni_munic['IBGE'].astype(str).str.zfill(6)

# Remover o dígito verificador (último dígito) da coluna 'Código Município Completo' no DataFrame 'municipios'
df_municipios['Código Município Completo'] = df_municipios['Código Município Completo'].astype(str).str[:-1]

# Padronizar para seis dígitos
df_municipios['Código Município Completo'] = df_municipios['Código Município Completo'].str.zfill(6)

# Realizar o merge entre os DataFrames
df_qtd_uni_municipios = pd.merge(
    df_qtd_uni_munic,
    df_municipios[['Nome_UF', 'Código Município Completo', 'Nome_Município']],
    left_on='IBGE',
    right_on='Código Município Completo',
    how='inner'
)

# Selecionar as colunas desejadas
df_qtd_uni_municipios = df_qtd_uni_municipios[['IBGE', 'Nome_Município', 'Nome_UF', 'qtd_pessoas_acolhidas', 'qtd_masc_total', 'qtd_fem_total']]

# Exibir o resultado
print("Resultado do merge:")
print(df_qtd_uni_municipios.head())

Resultado do merge:
     IBGE         Nome_Município   Nome_UF  qtd_pessoas_acolhidas  \
0  110001  Alta Floresta D'Oeste  Rondônia                      6   
1  110002              Ariquemes  Rondônia                     66   
2  110003                 Cabixi  Rondônia                      4   
3  110004                 Cacoal  Rondônia                     91   
4  110005             Cerejeiras  Rondônia                      4   

   qtd_masc_total  qtd_fem_total  
0               4              2  
1              39             27  
2               2              2  
3              67             24  
4               3              1  


#### Criar um dataframe que agrupa as quantidades total e de gênero de pessoas acolhidas por município no programa Família Acolhedora

In [50]:
df_qtd_total_fam = df_pessoas_fam_acolhedora.groupby('IBGE', as_index=False)[['qtd_criancas_acolhidas', 'qtd_masc_total', 'qtd_fem_total']].sum()

In [51]:
df_qtd_total_fam.head()

,IBGE,qtd_criancas_acolhidas,qtd_masc_total,qtd_fem_total
0,17,3,1,2
1,23,0,0,0
2,25,8,3,5
3,52,0,0,0
4,110011,0,0,0


In [52]:
df_qtd_total_fam_munic = df_qtd_total_fam[df_qtd_total_fam['IBGE'].between(100000, 999999)]

In [53]:
df_qtd_total_fam_munic.head()

,IBGE,qtd_criancas_acolhidas,qtd_masc_total,qtd_fem_total
4,110011,0,0,0
5,110012,5,4,1
6,110020,5,2,3
7,120040,9,0,9
8,130170,1,0,1


In [54]:
# Fazer uma cópia explícita do DataFrame
df_qtd_total_fam_munic = df_qtd_total_fam_munic.copy()

# Padronizar a coluna IBGE no DataFrame 'df_qtd_total_municipios' para ter seis dígitos
df_qtd_total_fam_munic['IBGE'] = df_qtd_total_fam_munic['IBGE'].astype(str).str.zfill(6)

# Realizar o merge entre os DataFrames
df_qtd_fam_municipios = pd.merge(
    df_qtd_total_fam_munic,
    df_municipios[['Nome_UF', 'Código Município Completo', 'Nome_Município']],
    left_on='IBGE',
    right_on='Código Município Completo',
    how='inner'
)

# Selecionar as colunas desejadas
df_qtd_fam_municipios = df_qtd_fam_municipios[['IBGE', 'Nome_Município', 'Nome_UF', 'qtd_criancas_acolhidas', 'qtd_masc_total', 'qtd_fem_total']]

# Exibir o resultado
print("Resultado do merge:")
print(df_qtd_fam_municipios.head())

Resultado do merge:
     IBGE Nome_Município   Nome_UF  qtd_criancas_acolhidas  qtd_masc_total  \
0  110011           Jaru  Rondônia                       0               0   
1  110012      Ji-Paraná  Rondônia                       5               4   
2  110020    Porto Velho  Rondônia                       5               2   
3  120040     Rio Branco      Acre                       9               0   
4  130170        Humaitá  Amazonas                       1               0   

   qtd_fem_total  
0              0  
1              1  
2              3  
3              9  
4              1  


#### Verificar shape e criar condições para fazer o merge dos dois dataframes

In [55]:
# Aqui dá pra perceber que a quantidade de municípios é diferente nos dois dataframes
print(df_qtd_uni_municipios.shape)
print(df_qtd_fam_municipios.shape)

(2278, 6)
(584, 6)


In [56]:
df_qtd_uni_municipios.columns

Index(['IBGE', 'Nome_Município', 'Nome_UF', 'qtd_pessoas_acolhidas',
       'qtd_masc_total', 'qtd_fem_total'],
      dtype='object')

In [57]:
df_qtd_fam_municipios.columns

Index(['IBGE', 'Nome_Município', 'Nome_UF', 'qtd_criancas_acolhidas',
       'qtd_masc_total', 'qtd_fem_total'],
      dtype='object')

In [58]:
# Realizar o merge
df_uni_fam_munic = pd.merge(df_qtd_uni_municipios, df_qtd_fam_municipios,
                        on=['IBGE', 'Nome_Município', 'Nome_UF'],
                        how='left',
                        suffixes=('_unidades', '_familia'))

In [59]:
df_uni_fam_munic.head()

,IBGE,Nome_Município,Nome_UF,qtd_pessoas_acolhidas,qtd_masc_total_unidades,qtd_fem_total_unidades,qtd_criancas_acolhidas,qtd_masc_total_familia,qtd_fem_total_familia
0,110001,Alta Floresta D'Oeste,Rondônia,6,4,2,NaN,NaN,NaN
1,110002,Ariquemes,Rondônia,66,39,27,NaN,NaN,NaN
2,110003,Cabixi,Rondônia,4,2,2,NaN,NaN,NaN
3,110004,Cacoal,Rondônia,91,67,24,NaN,NaN,NaN
4,110005,Cerejeiras,Rondônia,4,3,1,NaN,NaN,NaN


In [60]:
# Preencher valores nulos com zero
df_uni_fam_munic.fillna(0, inplace=True)

In [61]:
df_uni_fam_munic.head()

,IBGE,Nome_Município,Nome_UF,qtd_pessoas_acolhidas,qtd_masc_total_unidades,qtd_fem_total_unidades,qtd_criancas_acolhidas,qtd_masc_total_familia,qtd_fem_total_familia
0,110001,Alta Floresta D'Oeste,Rondônia,6,4,2,0.0,0.0,0.0
1,110002,Ariquemes,Rondônia,66,39,27,0.0,0.0,0.0
2,110003,Cabixi,Rondônia,4,2,2,0.0,0.0,0.0
3,110004,Cacoal,Rondônia,91,67,24,0.0,0.0,0.0
4,110005,Cerejeiras,Rondônia,4,3,1,0.0,0.0,0.0


#### Criar as variáveis pct_acolhidas_fam, pct_masc_acolhidos_fam e pct_fem_acolhidas_fam

In [62]:
# Criar a variável pct_acolhidas_fam: percentual total de pessoas acolhidas pelo Família Acolhedora
df_uni_fam_munic['pct_total_acolhidos_fam'] = df_uni_fam_munic['qtd_criancas_acolhidas'] / df_uni_fam_munic['qtd_pessoas_acolhidas']

In [63]:
# Criar a variável pct_masc_acolhidos_fam: percentual total de meninos/homens acolhidos pelo Família Acolhedora
df_uni_fam_munic['pct_masc_acolhidos_fam'] = df_uni_fam_munic['qtd_masc_total_familia'] / df_uni_fam_munic['qtd_masc_total_unidades']

In [64]:
# Criar a variável pct_fem_acolhidas_fam: percentual total de meninas/mulheres acolhidas pelo Família Acolhedora
df_uni_fam_munic['pct_fem_acolhidas_fam'] = df_uni_fam_munic['qtd_fem_total_familia'] / df_uni_fam_munic['qtd_fem_total_unidades']

In [65]:
# Substituir valores infinitos (resultado de divisão por zero) por zero
df_uni_fam_munic.replace([float('inf'), -float('inf')], 0, inplace=True)

In [66]:
df_uni_fam_munic.head()

,IBGE,Nome_Município,Nome_UF,qtd_pessoas_acolhidas,qtd_masc_total_unidades,qtd_fem_total_unidades,qtd_criancas_acolhidas,qtd_masc_total_familia,qtd_fem_total_familia,pct_total_acolhidos_fam,pct_masc_acolhidos_fam,pct_fem_acolhidas_fam
0,110001,Alta Floresta D'Oeste,Rondônia,6,4,2,0.0,0.0,0.0,0.0,0.0,0.0
1,110002,Ariquemes,Rondônia,66,39,27,0.0,0.0,0.0,0.0,0.0,0.0
2,110003,Cabixi,Rondônia,4,2,2,0.0,0.0,0.0,0.0,0.0,0.0
3,110004,Cacoal,Rondônia,91,67,24,0.0,0.0,0.0,0.0,0.0,0.0
4,110005,Cerejeiras,Rondônia,4,3,1,0.0,0.0,0.0,0.0,0.0,0.0


### Criação de gráficos para melhor visualização dos dados

#### Gráfico de barras empilhadas para visualizar das proporções

In [67]:
# Verificar se é possível entender que se a pessoa está acolhida no Programa Família Acolhedora ela continua a contar nas unidades de acolhimento?

#### Mapa com a média do percentual por UF

In [68]:
# Dados de coordenadas como lista de strings
dados_coordenadas = [
    {"uf": "AC", "latitude": -8.77, "longitude": -70.55},
    {"uf": "AL", "latitude": -9.62, "longitude": -36.82},
    {"uf": "AM", "latitude": -3.47, "longitude": -65.10},
    {"uf": "AP", "latitude": 1.41, "longitude": -51.77},
    {"uf": "BA", "latitude": -13.29, "longitude": -41.71},
    {"uf": "CE", "latitude": -5.20, "longitude": -39.53},
    {"uf": "DF", "latitude": -15.83, "longitude": -47.86},
    {"uf": "ES", "latitude": -19.19, "longitude": -40.34},
    {"uf": "GO", "latitude": -15.98, "longitude": -49.86},
    {"uf": "MA", "latitude": -5.42, "longitude": -45.44},
    {"uf": "MT", "latitude": -12.64, "longitude": -55.42},
    {"uf": "MS", "latitude": -20.51, "longitude": -54.54},
    {"uf": "MG", "latitude": -18.10, "longitude": -44.38},
    {"uf": "PA", "latitude": -3.79, "longitude": -52.48},
    {"uf": "PB", "latitude": -7.28, "longitude": -36.72},
    {"uf": "PR", "latitude": -24.89, "longitude": -51.55},
    {"uf": "PE", "latitude": -8.38, "longitude": -37.86},
    {"uf": "PI", "latitude": -6.60, "longitude": -42.28},
    {"uf": "RJ", "latitude": -22.25, "longitude": -42.66},
    {"uf": "RN", "latitude": -5.81, "longitude": -36.59},
    {"uf": "RO", "latitude": -10.83, "longitude": -63.34},
    {"uf": "RS", "latitude": -30.17, "longitude": -53.50},
    {"uf": "RR", "latitude": 1.99, "longitude": -61.33},
    {"uf": "SC", "latitude": -27.45, "longitude": -50.95},
    {"uf": "SE", "latitude": -10.57, "longitude": -37.45},
    {"uf": "SP", "latitude": -22.19, "longitude": -48.79},
    {"uf": "TO", "latitude": -9.46, "longitude": -48.26}
]

# Criar o DataFrame de coordenadas
df_coordenadas = pd.DataFrame(dados_coordenadas)

In [69]:
# Agrupar os dados por UF e calcular as somas
df_uf_acolhimento = df_uni_fam_munic.groupby('Nome_UF').agg({
    'qtd_pessoas_acolhidas': 'sum',
    'qtd_criancas_acolhidas': 'sum'
}).reset_index()

# Calcular a proporção total de acolhimento pelo Família Acolhedora para cada UF
df_uf_acolhimento['pct_total_acolhidos_fam'] = df_uf_acolhimento['qtd_criancas_acolhidas'] / df_uf_acolhimento['qtd_pessoas_acolhidas']

In [70]:
# Exibir os nomes únicos dos estados no seu dataframe
print(df_uf_acolhimento['Nome_UF'].unique())

['Acre' 'Alagoas' 'Amapá' 'Amazonas' 'Bahia' 'Ceará' 'Distrito Federal'
 'Espírito Santo' 'Goiás' 'Maranhão' 'Mato Grosso' 'Mato Grosso do Sul'
 'Minas Gerais' 'Paraná' 'Paraíba' 'Pará' 'Pernambuco' 'Piauí'
 'Rio Grande do Norte' 'Rio Grande do Sul' 'Rio de Janeiro' 'Rondônia'
 'Roraima' 'Santa Catarina' 'Sergipe' 'São Paulo' 'Tocantins']


In [71]:
# Remover espaços em branco extras e transformar em letras minúsculas para padronizar
df_uf_acolhimento['Nome_UF'] = df_uf_acolhimento['Nome_UF'].str.strip().str.title()

In [72]:
print(df_uf_acolhimento['Nome_UF'].unique())

['Acre' 'Alagoas' 'Amapá' 'Amazonas' 'Bahia' 'Ceará' 'Distrito Federal'
 'Espírito Santo' 'Goiás' 'Maranhão' 'Mato Grosso' 'Mato Grosso Do Sul'
 'Minas Gerais' 'Paraná' 'Paraíba' 'Pará' 'Pernambuco' 'Piauí'
 'Rio Grande Do Norte' 'Rio Grande Do Sul' 'Rio De Janeiro' 'Rondônia'
 'Roraima' 'Santa Catarina' 'Sergipe' 'São Paulo' 'Tocantins']


In [73]:
# Dicionário de conversão de nomes completos para siglas de estados
nome_para_sigla = {
    "Acre": "AC",
    "Alagoas": "AL",
    "Amapá": "AP",
    "Amazonas": "AM",
    "Bahia": "BA",
    "Ceará": "CE",
    "Distrito Federal": "DF",
    "Espírito Santo": "ES",
    "Goiás": "GO",
    "Maranhão": "MA",
    "Mato Grosso": "MT",
    "Mato Grosso Do Sul": "MS", # Adicionado com capitalização diferente
    "Minas Gerais": "MG",
    "Pará": "PA",
    "Paraíba": "PB",
    "Paraná": "PR",
    "Pernambuco": "PE",
    "Piauí": "PI",
    "Rio De Janeiro": "RJ",  # Adicionado com capitalização diferente
    "Rio Grande Do Norte": "RN",  # Adicionado com capitalização diferente
    "Rio Grande Do Sul": "RS",  # Adicionado com capitalização diferente
    "Rondônia": "RO",
    "Roraima": "RR",
    "Santa Catarina": "SC",
    "São Paulo": "SP",
    "Sergipe": "SE",
    "Tocantins": "TO"
}

In [74]:
df_uf_acolhimento['Nome_UF'] = df_uf_acolhimento['Nome_UF'].astype(str)
df_coordenadas['uf'] = df_coordenadas['uf'].astype(str)

In [75]:
# Adicionar a coluna de siglas ao dataframe de acolhimento
df_uf_acolhimento['uf'] = df_uf_acolhimento['Nome_UF'].map(nome_para_sigla)

In [76]:
# Adicionar as coordenadas
df_uf_acolhimento = df_uf_acolhimento.merge(df_coordenadas, on="uf", how="left")

In [77]:
df_uf_acolhimento

,Nome_UF,qtd_pessoas_acolhidas,qtd_criancas_acolhidas,pct_total_acolhidos_fam,uf,latitude,longitude
0,Acre,680,9.0,0.013235,AC,-8.77,-70.55
1,Alagoas,1426,0.0,0.000000,AL,-9.62,-36.82
2,Amapá,185,1.0,0.005405,AP,1.41,-51.77
3,Amazonas,607,36.0,0.059308,AM,-3.47,-65.10
4,Bahia,5958,16.0,0.002685,BA,-13.29,-41.71
5,Ceará,2077,29.0,0.013962,CE,-5.20,-39.53
6,Distrito Federal,1660,28.0,0.016867,DF,-15.83,-47.86
7,Espírito Santo,2644,18.0,0.006808,ES,-19.19,-40.34
8,Goiás,6132,8.0,0.001305,GO,-15.98,-49.86
9,Maranhão,538,11.0,0.020446,MA,-5.42,-45.44


In [78]:
# Criar o mapa centrado no Brasil
mapa_uf = folium.Map(location=[-14.235, -51.9253], zoom_start=4)

# Adicionar círculos para cada UF com o percentual de acolhidos
for _, row in df_uf_acolhimento.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=f"{row['Nome_UF']}: {row['pct_total_acolhidos_fam'] * 100:.2f}%"  # Exibir como porcentagem
    ).add_to(mapa_uf)

# Exibir o mapa
mapa_uf

#### Mapa com os percentuais dos municípios

In [80]:
df_municipios_lat_lon = pd.read_csv('../data/dados_geo/municipios.csv')

In [81]:
df_municipios_lat_lon.head()

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo


In [82]:
df_municipios_lat_lon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   codigo_ibge   5570 non-null   int64  
 1   nome          5570 non-null   object 
 2   latitude      5570 non-null   float64
 3   longitude     5570 non-null   float64
 4   capital       5570 non-null   int64  
 5   codigo_uf     5570 non-null   int64  
 6   siafi_id      5570 non-null   int64  
 7   ddd           5570 non-null   int64  
 8   fuso_horario  5570 non-null   object 
dtypes: float64(2), int64(5), object(2)
memory usage: 391.8+ KB


In [83]:
df_municipios_lat_lon['codigo_ibge'] = df_municipios_lat_lon['codigo_ibge'].astype(str).str[:-1].astype(int)

In [84]:
df_uni_fam_munic.head()

,IBGE,Nome_Município,Nome_UF,qtd_pessoas_acolhidas,qtd_masc_total_unidades,qtd_fem_total_unidades,qtd_criancas_acolhidas,qtd_masc_total_familia,qtd_fem_total_familia,pct_total_acolhidos_fam,pct_masc_acolhidos_fam,pct_fem_acolhidas_fam
0,110001,Alta Floresta D'Oeste,Rondônia,6,4,2,0.0,0.0,0.0,0.0,0.0,0.0
1,110002,Ariquemes,Rondônia,66,39,27,0.0,0.0,0.0,0.0,0.0,0.0
2,110003,Cabixi,Rondônia,4,2,2,0.0,0.0,0.0,0.0,0.0,0.0
3,110004,Cacoal,Rondônia,91,67,24,0.0,0.0,0.0,0.0,0.0,0.0
4,110005,Cerejeiras,Rondônia,4,3,1,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
# Converter ambas as colunas para int
df_uni_fam_munic['IBGE'] = df_uni_fam_munic['IBGE'].astype(int)
df_municipios_lat_lon['codigo_ibge'] = df_municipios_lat_lon['codigo_ibge'].astype(int)

In [86]:
df_uni_fam_munic.head()

,IBGE,Nome_Município,Nome_UF,qtd_pessoas_acolhidas,qtd_masc_total_unidades,qtd_fem_total_unidades,qtd_criancas_acolhidas,qtd_masc_total_familia,qtd_fem_total_familia,pct_total_acolhidos_fam,pct_masc_acolhidos_fam,pct_fem_acolhidas_fam
0,110001,Alta Floresta D'Oeste,Rondônia,6,4,2,0.0,0.0,0.0,0.0,0.0,0.0
1,110002,Ariquemes,Rondônia,66,39,27,0.0,0.0,0.0,0.0,0.0,0.0
2,110003,Cabixi,Rondônia,4,2,2,0.0,0.0,0.0,0.0,0.0,0.0
3,110004,Cacoal,Rondônia,91,67,24,0.0,0.0,0.0,0.0,0.0,0.0
4,110005,Cerejeiras,Rondônia,4,3,1,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
# Realizar o merge
df_uni_fam_munic = df_uni_fam_munic.merge(
    df_municipios_lat_lon[['codigo_ibge', 'latitude', 'longitude']],
    how='left', left_on='IBGE', right_on='codigo_ibge'
)

In [88]:
df_uni_fam_munic.head()

,IBGE,Nome_Município,Nome_UF,qtd_pessoas_acolhidas,qtd_masc_total_unidades,qtd_fem_total_unidades,qtd_criancas_acolhidas,qtd_masc_total_familia,qtd_fem_total_familia,pct_total_acolhidos_fam,pct_masc_acolhidos_fam,pct_fem_acolhidas_fam,codigo_ibge,latitude,longitude
0,110001,Alta Floresta D'Oeste,Rondônia,6,4,2,0.0,0.0,0.0,0.0,0.0,0.0,110001,-11.92830,-61.9953
1,110002,Ariquemes,Rondônia,66,39,27,0.0,0.0,0.0,0.0,0.0,0.0,110002,-9.90571,-63.0325
2,110003,Cabixi,Rondônia,4,2,2,0.0,0.0,0.0,0.0,0.0,0.0,110003,-13.49450,-60.5520
3,110004,Cacoal,Rondônia,91,67,24,0.0,0.0,0.0,0.0,0.0,0.0,110004,-11.43430,-61.4562
4,110005,Cerejeiras,Rondônia,4,3,1,0.0,0.0,0.0,0.0,0.0,0.0,110005,-13.18700,-60.8168


In [89]:
print(df_uni_fam_munic.isna().sum())

IBGE                         0
Nome_Município               0
Nome_UF                      0
qtd_pessoas_acolhidas        0
qtd_masc_total_unidades      0
qtd_fem_total_unidades       0
qtd_criancas_acolhidas       0
qtd_masc_total_familia       0
qtd_fem_total_familia        0
pct_total_acolhidos_fam     67
pct_masc_acolhidos_fam     147
pct_fem_acolhidas_fam      146
codigo_ibge                  0
latitude                     0
longitude                    0
dtype: int64


In [90]:
df_uni_fam_munic = df_uni_fam_munic.fillna(0)

In [91]:
import folium
from folium.plugins import HeatMap

# Criar o mapa centrado no Brasil
mapa_calor = folium.Map(location=[-14.235, -51.9253], zoom_start=4)

# Preparar os dados para o mapa de calor
heat_data = [
    [row['latitude'], row['longitude'], row['pct_total_acolhidos_fam']]
    for _, row in df_uni_fam_munic.iterrows()
]

# Adicionar o HeatMap ao mapa
HeatMap(heat_data, radius=15, max_zoom=10).add_to(mapa_calor)

# Adicionar uma legenda personalizada ao mapa
legend_html = '''
     <div style="
     position: fixed;
     bottom: 50px;
     left: 50px;
     width: 200px;
     height: 120px;
     background-color: white;
     border:2px solid grey;
     z-index:9999;
     font-size:14px;
     ">
     <h4 style="margin:5px">Intensidade do Acolhimento</h4>
     <p style="margin:5px">Baixa Intensidade: Azul claro</p>
     <p style="margin:5px">Alta Intensidade: Vermelho escuro</p>
     </div>
     '''

mapa_calor.get_root().html.add_child(folium.Element(legend_html))

# Exibir o mapa
mapa_calor